In [63]:
import pandas as pd
import sqlalchemy as db
import psycopg2 
import random

# Connect to the database via SQLalchemy
engine = db.create_engine('postgresql://{user}:{user_pass}@{host}/{dataname1}')

# Connect via psycopg2
conn = psycopg2.connect('dbname=dataname1 user={user} password={user_pass}')
cur = conn.cursor()

In [64]:
# Create a test data frame
df = pd.DataFrame(columns = ["id", "phone_request_needed", "phone_request_made"], index = range(0, 2000))
for i in range(0, 2000): 
    df.iloc[i] = [i, random.randint(0, 1), random.randint(0, 1)]
# Put into Postgresql
df.to_sql('test_requests', engine)

In [66]:
# Select a limited number of rows
working_table = pd.read_sql('SELECT * FROM test_requests '
                            'WHERE phone_request_made = 0 AND phone_request_needed = 1 LIMIT 100', 
                            engine)

In [67]:
working_table

,index,id,phone_request_needed,phone_request_made
0,1,1,1,0
1,7,7,1,0
2,9,9,1,0
3,13,13,1,0
4,14,14,1,0
...,...,...,...,...
95,385,385,1,0
96,388,388,1,0
97,393,393,1,0
98,401,401,1,0


In [68]:
assert (sum(working_table['phone_request_made'] == 1) == 0)

In [69]:
# Update rows 
working_table['phone_request_made'] = 1

In [71]:
# Update the original table
cur.execute("""CREATE TEMP TABLE updated_rows(id INTEGER, phone_request_made INTEGER) ON COMMIT DROP""")
rows = zip(working_table.id, working_table.phone_request_made)
cur.executemany("""INSERT INTO updated_rows (id, phone_request_made) VALUES(%s, %s)""", rows)
cur.execute("""
    UPDATE test_requests
    SET phone_request_made = updated_rows.phone_request_made
    FROM updated_rows
    WHERE updated_rows.id = test_requests.id;
    """)
conn.commit()

In [73]:
# See the changes
pd.read_sql('SELECT * FROM test_requests', engine)

,index,id,phone_request_needed,phone_request_made
0,0,0,0,0
1,2,2,0,0
2,3,3,1,1
3,4,4,0,1
4,5,5,0,0
...,...,...,...,...
1995,385,385,1,1
1996,388,388,1,1
1997,393,393,1,1
1998,401,401,1,1


In [74]:
# Close connection
engine.dispose()
conn.close()

In [4]:
import pandas as pd
import random
df = pd.DataFrame(columns = ["id", "phone_request_needed", "phone_request_made"], index = range(0, 10))
for i in range(0, 10): 
    df.iloc[i] = [i, random.randint(0, 1), random.randint(0, 1)]
df

,id,phone_request_needed,phone_request_made
0,0,0,1
1,1,0,1
2,2,1,1
3,3,1,0
4,4,1,0
5,5,0,0
6,6,1,0
7,7,0,1
8,8,0,0
9,9,1,0


In [37]:
def test_func(row):
    temp = [{'id': row['id'], 'a': 4, 'b': 2},
            {'id': row['id'], 'a': 1, 'b': 3}
           ]
    return pd.DataFrame(temp)

In [40]:
list(df.apply(test_func, axis = 1))

,id,a,b
0,1,4,2
1,1,1,3


In [ ]:
api_key = '***REMOVED***'
